# F1 Undercut Simulation Analysis

This notebook demonstrates how to use the F1 Undercut Simulator for advanced strategy analysis.

## Setup

In [ ]:
import sys
import os
sys.path.append('../backend')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Import our models
from models.deg import TireDegradationModel
from models.pit import PitStopModel
from models.outlap import OutlapModel
from services.openf1 import OpenF1Service

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

## 1. Tire Degradation Analysis

Let's analyze tire degradation patterns for different compounds.

In [ ]:
# Initialize the tire degradation model
deg_model = TireDegradationModel()

# Sample session data
session_data = {
    "lap_times": [90.5, 90.7, 90.9, 91.1, 91.4, 91.7, 92.0, 92.4, 92.8, 93.3],
    "tire_compounds": ["SOFT", "MEDIUM", "HARD"],
    "track_temperature": 35.0
}

# Run analysis
analysis = deg_model.analyze(session_data)
print(f"Track Temperature: {analysis['track_temperature']}°C")
print(f"Number of compounds analyzed: {len(analysis['analyses'])}")

In [ ]:
# Visualize tire degradation curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot performance curves
for analysis_result in analysis['analyses']:
    compound = analysis_result['compound']
    performance = analysis_result['predicted_performance']
    laps = range(1, len(performance) + 1)
    
    ax1.plot(laps, performance, marker='o', label=f'{compound} Compound', linewidth=2)

ax1.set_xlabel('Lap Number')
ax1.set_ylabel('Performance (%)')
ax1.set_title('Tire Performance Degradation')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot degradation rates
compounds = [a['compound'] for a in analysis['analyses']]
degradation_rates = [a['degradation_per_lap'] for a in analysis['analyses']]

bars = ax2.bar(compounds, degradation_rates, color=['red', 'yellow', 'white'], 
               edgecolor='black', alpha=0.7)
ax2.set_xlabel('Tire Compound')
ax2.set_ylabel('Degradation Rate (s/lap)')
ax2.set_title('Degradation Rate by Compound')

# Add value labels on bars
for bar, rate in zip(bars, degradation_rates):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
             f'{rate:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 2. Pit Strategy Optimization

Now let's explore different pit strategy options.

In [ ]:
# Initialize pit strategy model
pit_model = PitStopModel()

# Define race scenario
race_scenario = {
    "current_position": 6,
    "current_lap": 18,
    "total_laps": 55,
    "current_tire": "MEDIUM",
    "tire_age": 15,
    "fuel_load": 40.0,
    "weather": "dry",
    "traffic_density": 0.7
}

# Optimize strategy
strategy_result = pit_model.optimize(race_scenario)

print("Current Race Conditions:")
for key, value in strategy_result['current_conditions'].items():
    print(f"  {key}: {value}")

print("\nRecommended Strategy:")
recommended = strategy_result['recommended_strategy']
print(f"  Type: {recommended['strategy_type']}")
print(f"  Expected Position Gain: +{recommended['expected_position_gain']:.1f}")
print(f"  Confidence: {recommended['confidence']:.1%}")
print(f"  Risk Factor: {recommended['risk_factor']:.1f}")
print(f"  Description: {recommended['description']}")

In [ ]:
# Compare all strategy options
all_strategies = [strategy_result['recommended_strategy']] + strategy_result['alternative_strategies']

strategy_df = pd.DataFrame({
    'Strategy': [s['strategy_type'] for s in all_strategies],
    'Position Gain': [s['expected_position_gain'] for s in all_strategies],
    'Confidence': [s['confidence'] for s in all_strategies],
    'Risk Factor': [s['risk_factor'] for s in all_strategies]
})

print("Strategy Comparison:")
print(strategy_df.round(3))

# Visualize strategy comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Position gain vs risk
scatter = ax1.scatter(strategy_df['Risk Factor'], strategy_df['Position Gain'], 
                     s=strategy_df['Confidence']*500, alpha=0.7, c=range(len(strategy_df)))

for i, strategy in enumerate(strategy_df['Strategy']):
    ax1.annotate(strategy, (strategy_df['Risk Factor'].iloc[i], strategy_df['Position Gain'].iloc[i]),
                xytext=(5, 5), textcoords='offset points')

ax1.set_xlabel('Risk Factor')
ax1.set_ylabel('Expected Position Gain')
ax1.set_title('Strategy Risk vs Reward\n(Bubble size = Confidence)')
ax1.grid(True, alpha=0.3)

# Strategy performance radar chart
ax2.bar(strategy_df['Strategy'], strategy_df['Position Gain'], alpha=0.7)
ax2.set_xlabel('Strategy Type')
ax2.set_ylabel('Expected Position Gain')
ax2.set_title('Position Gain by Strategy')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 3. Outlap Performance Analysis

Let's analyze outlap performance for different tire compounds.

In [ ]:
# Initialize outlap model
outlap_model = OutlapModel()

# Compare outlap performance across compounds
session_key = "9158"  # Example session
fuel_load = 50.0

comparison = outlap_model.compare_compounds(
    session_key=session_key,
    fuel_load=fuel_load,
    baseline_lap_time=89.5,
    track_temp=32.0,
    lap_number=25
)

print("Outlap Performance Comparison:")
print(f"Fuel Load: {fuel_load}%")
print(f"Ranking (fastest to slowest): {comparison['ranking']}")
print(f"Time difference: {comparison['time_difference']:.3f}s")

# Create detailed comparison DataFrame
outlap_data = []
for compound, prediction in comparison['compound_comparison'].items():
    outlap_data.append({
        'Compound': compound,
        'Predicted Lap Time': prediction['predicted_lap_time'],
        'Confidence': prediction['confidence'],
        'Warm-up Laps': prediction['tire_warm_up_laps'],
        'Fuel Effect': prediction['fuel_effect'],
        'Total Penalty': prediction['total_penalty']
    })

outlap_df = pd.DataFrame(outlap_data)
print("\nDetailed Outlap Analysis:")
print(outlap_df.round(3))

In [ ]:
# Visualize outlap performance
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Lap time comparison
colors = ['red', 'yellow', 'lightgray']
bars1 = ax1.bar(outlap_df['Compound'], outlap_df['Predicted Lap Time'], 
                color=colors, alpha=0.7, edgecolor='black')
ax1.set_xlabel('Tire Compound')
ax1.set_ylabel('Predicted Lap Time (s)')
ax1.set_title('Outlap Performance by Compound')

# Add time labels
for bar, time in zip(bars1, outlap_df['Predicted Lap Time']):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{time:.3f}s', ha='center', va='bottom')

# Confidence levels
bars2 = ax2.bar(outlap_df['Compound'], outlap_df['Confidence'], 
                color=colors, alpha=0.7, edgecolor='black')
ax2.set_xlabel('Tire Compound')
ax2.set_ylabel('Prediction Confidence')
ax2.set_title('Confidence by Compound')
ax2.set_ylim(0, 1)

# Warm-up characteristics
bars3 = ax3.bar(outlap_df['Compound'], outlap_df['Warm-up Laps'], 
                color=colors, alpha=0.7, edgecolor='black')
ax3.set_xlabel('Tire Compound')
ax3.set_ylabel('Warm-up Laps')
ax3.set_title('Tire Warm-up Time')

# Performance factors breakdown
factors = ['Fuel Effect', 'Total Penalty']
x = np.arange(len(outlap_df))
width = 0.35

ax4.bar(x - width/2, outlap_df['Fuel Effect'], width, label='Fuel Effect', alpha=0.7)
ax4.bar(x + width/2, outlap_df['Total Penalty'], width, label='Total Penalty', alpha=0.7)

ax4.set_xlabel('Tire Compound')
ax4.set_ylabel('Time Penalty (s)')
ax4.set_title('Performance Impact Factors')
ax4.set_xticks(x)
ax4.set_xticklabels(outlap_df['Compound'])
ax4.legend()

plt.tight_layout()
plt.show()

## 4. Integrated Strategy Analysis

Let's combine all models for a comprehensive strategy analysis.

In [ ]:
# Simulate a complete race scenario
def simulate_race_scenario(current_lap, total_laps, current_position):
    """
    Simulate a complete race scenario with all models.
    """
    # Tire degradation analysis
    session_data = {
        "lap_times": [89.5 + i*0.1 for i in range(15)],  # Simulated degrading lap times
        "tire_compounds": ["SOFT", "MEDIUM", "HARD"],
        "track_temperature": 28.0
    }
    degradation = deg_model.analyze(session_data)
    
    # Pit strategy optimization
    strategy_request = {
        "current_position": current_position,
        "current_lap": current_lap,
        "total_laps": total_laps,
        "current_tire": "MEDIUM",
        "tire_age": current_lap - 1,
        "fuel_load": max(10, 100 - (current_lap / total_laps) * 90),
        "weather": "dry",
        "traffic_density": 0.6
    }
    strategy = pit_model.optimize(strategy_request)
    
    # Outlap performance prediction
    recommended_compound = strategy['recommended_strategy']['pit_windows'][0]['tire_compound']
    outlap = outlap_model.predict(
        session_key="sim",
        tire_compound=recommended_compound,
        fuel_load=strategy_request['fuel_load'],
        baseline_lap_time=89.5
    )
    
    return {
        'degradation': degradation,
        'strategy': strategy,
        'outlap': outlap
    }

# Run simulation
scenario = simulate_race_scenario(current_lap=20, total_laps=58, current_position=7)

print("Integrated Race Strategy Analysis")
print("=" * 40)
print(f"Track Temperature: {scenario['degradation']['track_temperature']}°C")
print(f"Recommended Strategy: {scenario['strategy']['recommended_strategy']['strategy_type']}")
print(f"Expected Position Gain: +{scenario['strategy']['recommended_strategy']['expected_position_gain']:.1f}")
print(f"Outlap Compound: {scenario['outlap']['tire_compound']}")
print(f"Predicted Outlap Time: {scenario['outlap']['prediction']['predicted_lap_time']:.3f}s")
print(f"Outlap Confidence: {scenario['outlap']['prediction']['confidence']:.1%}")

## 5. Sensitivity Analysis

Let's analyze how different factors affect strategy decisions.

In [ ]:
# Analyze sensitivity to track temperature
temperatures = range(15, 46, 5)  # 15°C to 45°C
temp_results = []

for temp in temperatures:
    session_data = {
        "lap_times": [89.5 + i*0.1 for i in range(10)],
        "tire_compounds": ["SOFT", "MEDIUM", "HARD"],
        "track_temperature": float(temp)
    }
    
    analysis = deg_model.analyze(session_data)
    
    # Extract degradation rates
    for compound_analysis in analysis['analyses']:
        temp_results.append({
            'Temperature': temp,
            'Compound': compound_analysis['compound'],
            'Degradation_Rate': compound_analysis['degradation_per_lap']
        })

temp_df = pd.DataFrame(temp_results)

# Plot temperature sensitivity
plt.figure(figsize=(12, 8))

for compound in ['SOFT', 'MEDIUM', 'HARD']:
    compound_data = temp_df[temp_df['Compound'] == compound]
    plt.plot(compound_data['Temperature'], compound_data['Degradation_Rate'], 
             marker='o', linewidth=2, label=f'{compound} Compound')

plt.xlabel('Track Temperature (°C)')
plt.ylabel('Degradation Rate (s/lap)')
plt.title('Tire Degradation Sensitivity to Track Temperature')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Temperature Sensitivity Analysis:")
print("- Soft tires show highest degradation rates")
print("- All compounds show increased degradation at extreme temperatures")
print("- Hard tires are most stable across temperature range")

## 6. Summary and Insights

Based on our analysis, here are the key insights:

In [ ]:
print("F1 Undercut Simulation - Key Insights")
print("=" * 50)
print()
print("1. TIRE DEGRADATION:")
print("   • Soft compounds offer peak performance but degrade quickly")
print("   • Medium compounds provide good balance of speed and durability")
print("   • Hard compounds last longest but sacrifice initial pace")
print()
print("2. PIT STRATEGY:")
print("   • Two-stop strategies often provide best position gain potential")
print("   • Undercut opportunities depend heavily on traffic density")
print("   • Risk vs reward must be carefully balanced")
print()
print("3. OUTLAP PERFORMANCE:")
print("   • Fresh tire performance varies significantly by compound")
print("   • Fuel load has measurable impact on outlap times")
print("   • Track temperature affects tire warm-up characteristics")
print()
print("4. STRATEGIC RECOMMENDATIONS:")
print("   • Monitor track temperature for optimal compound selection")
print("   • Consider traffic when planning undercut attempts")
print("   • Balance aggressive strategy with confidence levels")
print()
print("This analysis framework can be used to:")
print("• Optimize real-time race strategies")
print("• Analyze historical race data")
print("• Compare different strategic approaches")
print("• Predict outcomes of strategic decisions")